# Count eggs using roboflow trained roboflow 3.0

## Import packages

In [ ]:
import os
import supervision as sv
from inference import get_roboflow_model
import torch
import onnxruntime as rt
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import multiprocessing as mp

## Define functions

In [3]:
def callback(image_slice: np.ndarray) -> sv.Detections:
    results = model.infer(image_slice, confidence=0.45, iou_threshold=0.5)[0] # Reset confidence level here
    return sv.Detections.from_inference(results)

In [4]:
def count_eggs(image_path):
    # Read image path
    image = cv2.imread(image_path)

    # Decode the QR code in the image
    qcd = cv2.QRCodeDetector()

    retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(image)
    points

    if not retval:
        decoded_info = [os.path.basename(image_path)]
    else:
        # Calculate the center of the square
        center = np.mean(points, axis=1)

        # Expand the points by the specified factors
        scale_x = 1.6
        scale_y = 2.25
        expanded_points = np.copy(points)
        expanded_points[:, :, 0] = scale_x * (points[:, :, 0] - center[:, 0]) + center[:, 0]
        expanded_points[:, :, 1] = scale_y * (points[:, :, 1] - center[:, 1]) + center[:, 1]

        # Convert points to integer coordinates
        expanded_points = expanded_points.astype(int)

        # Create a mask with the same dimensions as the image
        mask = np.ones(image.shape[:2], dtype=np.uint8) * 255

        # Fill the mask with the expanded points
        cv2.fillPoly(mask, expanded_points, 0)

        # Apply the mask to the image to remove the specified points
        image = cv2.bitwise_and(image, image, mask=mask)
    
    # Slice Image and run inference using callback function
    slicer = sv.InferenceSlicer(callback=callback, slice_wh=(640, 640))
    sliced_detections = slicer(image=image)

    # Save output image
    box_annotator = sv.BoxAnnotator()
    # You can also use sv.MaskAnnotator() for instance segmentation models
    # mask_annotator = sv.MaskAnnotator()
    annotated_image = box_annotator.annotate(
        scene=image.copy(), detections=sliced_detections)
    # Convert the annotated image from numpy array to PIL Image
    image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    annotated_image_pil = Image.fromarray(image_rgb)
    annotated_image_resized = annotated_image_pil.resize((4000, 6000))
    annotated_image_resized.save(f'User/path/to/images/{decoded_info[0]}.png')

    # Save information to create a pandas df for export
    class_names = sliced_detections['class_name']
    class_id = sliced_detections.class_id
    confidence = sliced_detections.confidence

    # save objects into a single df
    df = (pd.DataFrame({'ID': class_names, 'class_id': class_id, 'confidence': confidence})
        .value_counts(subset = 'ID')
        .to_frame(decoded_info[0])
        .T.assign(total_eggs = len(sliced_detections)))
    
    return df


In [ ]:
# Load images
directory = "User/path/to/images"
image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.JPG')]

len(image_paths)

In [ ]:
model = get_roboflow_model(model_id="egg_training-bi/1", api_key="l6XPyOniqM4Ecq129cpf")

For loop run

In [ ]:
# Multi Image run for model
results = []

# Process images sequentially using a for loop
for image_path in image_paths:
    try:
        result = count_eggs(image_path)
        results.append(result)
        print(f"Processed {image_path}")
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        continue

Multithread run

In [ ]:
def process_image(image_path):
    try:
        result = count_eggs(image_path)
        return result
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

if __name__ == "__main__":
    # List of image paths
    directory = "/Users/aja294/Library/CloudStorage/Box-Box/Breeding Insight/Species/Salmonids/Idaho Group/Trout/Image Analysis/final_images_2024/eggs"
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.JPG')]

    # Number of worker processes
    num_workers = mp.cpu_count() // 2  # Use half of the available CPU cores

    # Create a pool of worker processes
    with mp.Pool(num_workers) as pool:
        # Map the image paths to the process_image function
        results = pool.map(process_image, image_paths)

    # Filter out None results (in case of errors)
    results = [result for result in results if result is not None]

    # Print the results
    print("Results:", results)

Combine results

In [ ]:
# Combine all DataFrames into a single DataFrame
final_df = pd.concat(results)

# Print the final DataFrame
print(final_df)

In [12]:
final_df.to_csv('result/path', index=True)